In [ ]:
import sys
import localSettings as ls
print(ls.main_path)

In [ ]:
main_path = ls.main_path
sys.path.append(main_path)

In [ ]:
SHRCALIBFACTOR = 1.00

In [ ]:
# Doing Run3?
ISRUN3 = False

In [ ]:
import plotter
import importlib
importlib.reload(plotter)
import uproot
import matplotlib.pylab as pylab
import numpy as np
import math
from sklearn.model_selection import train_test_split
import pickle
import xgboost as xgb
import nue_booster 
importlib.reload(nue_booster)
import awkward

params = {
    'axes.labelsize': 'x-large',
    'axes.titlesize': 'x-large',
    'xtick.labelsize': 'x-large',
    'ytick.labelsize': 'x-large'
}
pylab.rcParams.update(params)

In [ ]:
fold = "nuselection"
tree = "NeutrinoSelectionFilter"

BNB = ""
EXT = ""
NU  = ""
NUE = ""
DRT = ""
PI0 = ""
CCPI0 = ""

if ISRUN3:
    BNB = 'data_bnb_mcc9.1_v08_00_00_25_reco2_G1_beam_good_reco2_1e19'
    EXT = 'data_extbnb_mcc9.1_v08_00_00_25_reco2_G1_all_reco2'
    NU  = 'prodgenie_bnb_nu_uboone_overlay_mcc9.1_v08_00_00_26_filter_run3_reco2_G_reco2'
    NUE = 'prodgenie_bnb_intrinsice_nue_uboone_overlay_mcc9.1_v08_00_00_26_run3_reco2_reco2'
    DRT = 'prodgenie_bnb_dirt_overlay_mcc9.1_v08_00_00_26_run3_reco2_reco2'
    PI0 = 'prodgenie_nc_pi0_uboone_overlay_mcc9.1_v08_00_00_26_run3_G_reco2'
    CCPI0 = 'prodgenie_cc_pi0_uboone_overlay_v08_00_00_26_run3_G_reco2'
    
else:
    BNB = 'data_bnb_mcc9.1_v08_00_00_25_reco2_C1_beam_good_reco2_5e19'
    EXT = 'data_extbnb_mcc9.1_v08_00_00_25_reco2_C1_all_reco2'
    NU  = 'prodgenie_bnb_nu_uboone_overlay_mcc9.1_v08_00_00_26_filter_run1_reco2_reco2'
    NUE = 'prodgenie_bnb_intrinsice_nue_uboone_overlay_mcc9.1_v08_00_00_26_run1_reco2_reco2'
    DRT = 'prodgenie_bnb_dirt_overlay_mcc9.1_v08_00_00_26_run1_reco2_reco2'
    PI0 = 'prodgenie_nc_pi0_uboone_overlay-v08_00_00_26_run1_reco2_reco2'
    CCPI0 = 'prodgenie_cc_pi0_uboone_overlay_v08_00_00_26_run1_reco2'
    
mc = uproot.open(ls.ntuple_path+NU+".root")[fold][tree]
nc = uproot.open(ls.ntuple_path+PI0+".root")[fold][tree]
ccpi0 = uproot.open(ls.ntuple_path+CCPI0+".root")[fold][tree]
nue = uproot.open(ls.ntuple_path+NUE+".root")[fold][tree]
data = uproot.open(ls.ntuple_path+BNB+".root")[fold][tree]
ext = uproot.open(ls.ntuple_path+EXT+".root")[fold][tree]
dirt = uproot.open(ls.ntuple_path+DRT+".root")[fold][tree]
lee = uproot.open(ls.ntuple_path+NUE+".root")[fold][tree]

uproot_v = [lee,mc,nc,ccpi0,nue,ext,data,dirt]

variables = [
    "run","sub","evt",
    "shr_bkt_pdg", "selected", "nu_pdg",
    "slpdg", "trk_score_v", "backtracked_pdg", # modified from shr_score_v
    "shr_pfp_id_v", "category",
    "reco_nu_vtx_sce_x","reco_nu_vtx_sce_y","reco_nu_vtx_sce_z",
    "reco_nu_vtx_x","reco_nu_vtx_y","reco_nu_vtx_z",
    "nu_e","trk_bkt_pdg","ccnc",
    "trk_energy_muon",
    "nu_flashmatch_score","trk_phi",
    "trk_llr_pid_score_v","trk_id",
    "muon_e","proton_e","nproton",
    "mc_pdg", "npi0","interaction","NeutrinoEnergy2",
    "topological_score","trk_len","trk_theta",
    "evnunhits", "nslice", "selected",
    "reco_nu_vtx_x", "reco_nu_vtx_y", "reco_nu_vtx_z"]

nue = nue.pandas.df(variables + ["weightSpline","weightTune"], flatten=False)
mc = mc.pandas.df(variables + ["weightSpline","weightTune"], flatten=False)
nc = nc.pandas.df(variables + ["weightSpline","weightTune"], flatten=False)
ccpi0 = ccpi0.pandas.df(variables + ["weightSpline","weightTune"], flatten=False)
data = data.pandas.df(variables, flatten=False)
ext = ext.pandas.df(variables, flatten=False)
dirt = dirt.pandas.df(variables + ["weightSpline","weightTune"], flatten=False)
lee = lee.pandas.df(variables + ["weightSpline", "leeweight","weightTune"], flatten=False)

In [ ]:
df_v = [lee,mc,nc,ccpi0,nue,dirt]

for i,df in enumerate(df_v):
    
    df.loc[ df['weightTune'] < 0, 'weightTune' ] = 1.
    df.loc[ df['weightTune'] == np.inf, 'weightTune' ] = 1.
    df.loc[ df['weightTune'] > 100, 'weightTune' ] = 1.
    df['weightSpline']  = df['weightSpline']  * df['weightTune']

In [ ]:
def EQE(df,elecangle,elecenergy,ke):
    mp = 0.998
    mn = 0.939
    Eb = 0.00987 # see DocDB 26949
    theta = df[elecangle]
    El = df[elecenergy]
    if (ke == True):
        El += 0.105
    ml = 0.105
    num = mp**2 - (mn-Eb)**2 - ml**2 + 2*(mn-Eb)*El
    den = 2*(mn-Eb-El + np.sqrt(El**2-ml**2) * np.cos(theta))
    return num/den

In [ ]:
df_v = [lee,mc,nc,ccpi0,nue,ext,data,dirt]

for i,df in enumerate(df_v):
    up = uproot_v[i]
    trk_llr_pid_v = up.array('trk_llr_pid_score_v')
    trk_id = up.array('trk_id')-1 # I think we need this -1 to get the right result
    trk_llr_pid_v_sel = awkward.fromiter([pidv[tid] if tid<len(pidv) else 9999. for pidv,tid in zip(trk_llr_pid_v,trk_id)])
    df['trkpid'] = trk_llr_pid_v_sel
    df["reco_e_qe"] = EQE(df,'trk_theta','trk_energy_muon',1)

In [ ]:
# avoid double-counting of events out of FV in the NC/CC pi0 samples
nc = nc.query('category != 5')
ccpi0 = ccpi0.query('category != 5')

In [ ]:
df_v = [lee,mc,nc,ccpi0,nue,ext,data,dirt]

# and a way to filter out data
for i,df in enumerate(df_v):
    df["bnbdata"] = np.zeros_like(df["topological_score"])
data["bnbdata"] = np.ones_like(data["topological_score"])

In [ ]:
lee["is_signal"] = lee["category"] == 11
data["is_signal"] = data["category"] == 11
nue["is_signal"] = nue["category"] == 11
mc["is_signal"] = mc["category"] == 11
dirt["is_signal"] = dirt["category"] == 11
ext["is_signal"] = ext["category"] == 11
nc["is_signal"] = nc["category"] == 11
ccpi0["is_signal"] = ccpi0["category"] == 11

lee.loc[lee['category'] == 1, 'category'] = 111
lee.loc[lee['category'] == 10, 'category'] = 111
lee.loc[lee['category'] == 11, 'category'] = 111

In [ ]:
dfbdt_v = [lee,mc,nc,ccpi0,nue,ext,data,dirt]

In [ ]:
mc = mc.query('weightTune < 100.')

In [ ]:
samples = {
    "mc": mc,
    "nue": nue,
    "data": data,
    "ext": ext,
    "dirt": dirt,
    #"nc": nc,
    #"cc": ccpi0,
    "lee": lee
}

scaling = 1
#scaling = 132.0/0.486
#scaling = 132.0/3.9

SPLIT = 1.0

#''' 1115
if ISRUN3:
    weights = {
        "mc": 5.53e-3 * SPLIT * scaling,
        "nue": 1.26e-4 * SPLIT * scaling,
        "ext": 3.51e-2 * SPLIT * scaling,
        "dirt": 2.26e-2 * scaling,
        "lee": 1.26e-4 * SPLIT * scaling,
        "nc": 3.21e-3 * SPLIT * scaling,
        "cc": 1.35e-3 * SPLIT * scaling,
    }
    pot = 0.742e19*scaling
else:
    weights = {
        "mc": 6.33e-1 * SPLIT * scaling,
        "nue": 1.39e-3 * SPLIT * scaling,
        "ext": 2.87e-1 * SPLIT * scaling,
        "dirt": 3.01e-1 * scaling,
        "lee": 1.16e-2 * SPLIT * scaling,
        "nc": 3.15e-2 * SPLIT * scaling,
        "cc": 2.45e-2 * SPLIT * scaling,
    }
    pot = 4.10e19*scaling
#'''



my_plotter = plotter.Plotter(samples, weights, pot=pot)

In [ ]:
# muon selection
QUERY = 'nslice == 1'
#if ISRUN3: QUERY += ' and ((crtveto!=1) or (crthitpe < 100)) and (_closestNuCosmicDist > 20.)'
#QUERY += ' and trk_len > 20'
#QUERY += ' and topological_score > 0.06'
#QUERY += ' and reco_nu_vtx_sce_x > 5 and reco_nu_vtx_sce_x < 251'
#QUERY += ' and reco_nu_vtx_sce_y > -110 and reco_nu_vtx_sce_y < 110'
#QUERY += ' and reco_nu_vtx_sce_z > 20 and reco_nu_vtx_sce_z < 986'
#QUERY += ' and trkpid > 0.2'

print (QUERY)

In [ ]:
from datetime import datetime
now = datetime.now()
date_time = now.strftime("%m%d%Y")
print("date and time:",date_time)

In [ ]:
VARIABLE, BINS, RANGE, XTIT = 'topological_score',25,(0.,1.),"Slice Topological Score"

fig, ax1, ax2 = my_plotter.plot_variable(
    VARIABLE,   
    query=QUERY,
    kind="event_category",
    #kind="interaction",
    #kind="sample",
    #kind='particle_pdg',
    draw_sys=False,
    title=XTIT,
    #bins=asymm_bins,
    bins=BINS,
    range=RANGE,
)

#print("Profile likelihood: %.2f sigma @ 1.32e21 POT" % my_plotter.significance_likelihood)
#print("s/sqrt(b): %.2f sigma @ 1.32e21 POT" % my_plotter.significance)

#ax1.set_ylim(0,1200)
#ax1.set_yscale("log")
#ax1.set_ylim(0,0.25)
ax2.set_ylim(0.5,1.5)
#fig.savefig(ls.plots_path+VARIABLE+"_"+date_time+".pdf")

In [ ]:
EVTQUERY = QUERY + ' and reco_nu_vtx_sce_x > 100.35 and reco_nu_vtx_sce_x < 100.38'
mctmp = mc.query(EVTQUERY)
print (mctmp['weightTune'])
print (mctmp['ccnc'])
print (mctmp['interaction'])
print (mctmp['npi0'])
print (mctmp['run'])
print (mctmp['sub'])
print (mctmp['evt'])

In [ ]:
def Eff(df,var,query,acceptance,bin_edges,absval=False):
    #print acceptance
    bin_centers = 0.5*(bin_edges[1:]+bin_edges[:-1])
    bins = []
    bin_eff = []
    bin_err = []
    for i in range(len(bin_centers)):
        binmin = bin_edges[i]
        binmax = bin_edges[i+1]
        bincut = '%s > %f and %s < %f'%(var,binmin,var,binmax)
        if (absval == True):
            bincut = '(%s > %f and %s < %f) or (%s > -%f and %s < -%f)'%(var,binmin,var,binmax,var,binmax,var,binmin)
        if (acceptance != ''): bincut += ' and %s'%acceptance
        #print bincut
        df_tmp =  df.query(bincut) # cut on bin range for desired var.
        df_sub = df_tmp.query(query) # apply constrain 
        if (df_tmp.shape[0] == 0): continue
        eff = df_sub.shape[0] / float( df_tmp.shape[0] )
        err = np.sqrt( eff*(1-eff)/df_tmp.shape[0] )
        bin_eff.append( eff )
        bin_err.append( err )
        bins.append(bin_centers[i])
        #print 'eff = %.02f @ bin = %.02f'%(eff,bin_centers[i])
    return np.array(bins),np.array(bin_eff),np.array(bin_err)

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 12})

In [ ]:
# Run 1
RMIN = 4000
RMAX = 8000
# Run 3
#RMIN = 12000
#RMAX = 20000

In [ ]:
fig = plt.figure(figsize=(12,6))
B = np.linspace(RMIN,RMAX,50)

ACCEPTANCE = ''
VAR = 'run' #'_pi0_e'

for label,df in samples.items():
    
    if (label == "lee"):
        continue

    centers,vals,errs = Eff(df,VAR,QUERY,"",B)
    #popt, pcov = curve_fit(CONST,centers,vals,sigma=errs,absolute_sigma=True)
    plt.errorbar(centers,vals,yerr=errs,fmt='o-',label=r'%s'%(label))
    #print 'intrinsic nue pass-rage : %.03f'%(popt[0])
    #plt.axhline(popt[0],color='b')


plt.xlabel(r'Run Number')
plt.ylabel(r'SliceID')
plt.grid()
plt.legend(loc=1,fontsize=14,framealpha=0.5)
#plt.yscale('log')
plt.title(r'v08_00_00_26 Run1 Samples')
#plt.legend(loc=7,fontsize=14)
plt.show()